In [6]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def gibbs_sampling(v, W, h_bias):
    h_prob = sigmoid(np.dot(v, W) + h_bias)
    h = (np.random.rand(*h_prob.shape) < h_prob).astype(float)
    v_prob = sigmoid(np.dot(h, W.T) + v_bias)
    v_sample = (np.random.rand(*v_prob.shape) < v_prob).astype(float)
    return v_sample

def contrastive_divergence(v0, W, h_bias, k=1, learning_rate=0.1):
    v = v0
    v_bias = np.zeros(v.shape)
    h0_prob = sigmoid(np.dot(v, W) + h_bias)
    h0 = (np.random.rand(*h0_prob.shape) < h0_prob).astype(float)

    v_k = v
    for _ in range(k):
        v_k = gibbs_sampling(v_k, W, 0)  # 0 bias for visible layer during Gibbs sampling

    h_k_prob = sigmoid(np.dot(v_k, W) + h_bias)
    h_k = (np.random.rand(*h_k_prob.shape) < h_k_prob).astype(float)

    # Update weights and biases
    dW = np.outer(v0, h0) - np.outer(v_k, h_k)
    dv_bias = v0 - v_k
    dh_bias = h0 - h_k

    W += learning_rate * dW
    v_bias += learning_rate * dv_bias
    h_bias += learning_rate * dh_bias

# Example usage
# Define RBM parameters
num_visible = 4
num_hidden = 2
W = np.random.rand(num_visible, num_hidden)
v_bias = np.zeros(num_visible)
h_bias = np.zeros(num_hidden)

# Training data (binary values)
data = np.array([[1, 0, 1, 0],
                 [1, 1, 0, 0],
                 [0, 0, 1, 1]])

# Training using Contrastive Divergence
epochs = 1000
learning_rate = 0.1
for epoch in range(epochs):
    for v0 in data:
        contrastive_divergence(v0, W, h_bias, k=1, learning_rate=learning_rate)

# Testing
test_data = np.array([[1, 0, 0, 0],
                      [0, 1, 1, 0]])

for v0 in test_data:
    h_prob = sigmoid(np.dot(v0, W) + h_bias)
    h_sample = (np.random.rand(*h_prob.shape) < h_prob).astype(float)
    reconstructed_v = sigmoid(np.dot(h_sample, W.T) + v_bias)
    print("Original:", v0)
    print("Reconstructed:", (reconstructed_v > 0.5).astype(int))
    print()


Original: [1 0 0 0]
Reconstructed: [1 1 0 0]

Original: [0 1 1 0]
Reconstructed: [1 0 1 0]

